# LU разложение, блочные и ленточные матрицы

Существует множество типов матриц, удовлетворяющих некоторым дополнительным условиям, для которых многие матричные операции могут быть вычислены быстрее или точнее, чем для матриц произвольного вида. 
В данной лабораторной мы начнем писать библиотеку на Python, которая будет содержать классы, реализующие базовые алгоритмы для работы с основными типами матриц.
Далее приводится исходный код класса `Matrix`, являющегося общим предком для всех матриц, и реализующего логику работы с матрицами общего вида.
Нижеследующий класс `FullMatrix` реализует хранилище для заполненных матриц. 
Изучите эти реализации и выполните следующие задания:


7. Как влияет симметричность матрицы на устойчивость LU разложения?
8. Реализуйте класс `BandMatrix` для хранения ленточных матриц. Убедитесь в работоспособности методов `lu` и `solve`.
9. Воспользуйтесь реализованными классами для решения уравнения Пуассона $\Delta f=g$, использую операцию Лапласа из предыдущей лабораторной.

### Done

1. Напишите метод `lu` для класса `Matrix`, выполняющий LU разложение. 
2. Реализуйте метод `det`, вычисляющий определитель матрицы, опираясь на LU разложение.
3. Реализация `FullMatrix` может содержать своими элементами другие матрицы, т.е. описывать блочную матрицу. Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.3. Реализация `FullMatrix` может содержать своими элементами другие матрицы, т.е. описывать блочную матрицу. Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.
4. Реализуйте LUP разложение с перестановкой строк. Предъявите матрицу, на которой LUP разложение работает, а LU - нет.
5. Реализуйте метод прогонки и реализуйте метод `Matrix.solve` для решения линейных систем уравнений.
6. Реализуйте класс `SymmetricMatrix`, хранящий симметричные матрицы. Убедитесь, что метод `Matrix.lu` корректно работает с этим классом. Модифицируйте этот метод для класса `SymmetricMatrix` так, чтобы он использовал симметричность матрицы и работал в два раза быстрее.

In [161]:
import numpy as np
from fractions import Fraction
from numbers import Number

In [190]:
class TextBlock:
    def __init__(self, rows):
        assert isinstance(rows, list)
        self.rows = rows
        self.height = len(self.rows)
        self.width = max(map(len,self.rows))
        
    @classmethod
    def from_str(_cls, data):
        assert isinstance(data, str)
        return TextBlock( data.split('\n') )
        
    def format(self, width=None, height=None):
        if width is None: width = self.width
        if height is None: height = self.height
        return [f"{row:{width}}" for row in self.rows]+[' '*width]*(height-self.height)
    
    @staticmethod
    def merge(blocks):
        return [" ".join(row) for row in zip(*blocks)]

class Matrix:
    """Общий предок для всех матриц."""
    @property
    def shape(self):
        raise NotImplementedError
    
    @property
    def dtype(self):
        raise NotImplementedError
    
    @property 
    def width(self):
        return self.shape[1]
    
    @property 
    def height(self):
        return self.shape[0]    
        
    def __repr__(self):
        """Возвращает текстовое представление для матрицы."""
        text = [[TextBlock.from_str(f"{self[r,c]}") for c in range(self.width)] for r in range(self.height)]
        width_el = np.array(list(map(lambda row: list(map(lambda el: el.width, row)), text)))
        height_el = np.array(list(map(lambda row: list(map(lambda el: el.height, row)), text)))
        width_column = np.max(width_el, axis=0)
        width_total = np.sum(width_column)
        height_row = np.max(height_el, axis=1)
        result = []
        for r in range(self.height):
            lines = TextBlock.merge(text[r][c].format(width=width_column[c], height=height_row[r]) for c in range(self.width))
            for l in lines:
                result.append(f"| {l} |")
            if len(lines)>0 and len(lines[0])>0 and lines[0][0]=='|' and r<self.height-1:
                result.append(f'| {" "*(width_total+self.width)}|')
        return "\n".join(result)
    
    def empty_like(self, width=None, height=None):
        raise NotImplementedError
        
    def ident_like(self):
        raise NotImplementedError
    
    def __getitem__(self, key):
        raise NotImplementedError
    
    def __setitem__(self, key, value):
        raise NotImplementedError
        
    def __add__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] + other[r,c]
            return matrix
        return NotImplemented
    
    def __sub__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] - other[r,c]
            return matrix
        return NotImplemented

    def __mul__(self, other):
        return self.__matmul__(other)
    
    def __matmul__(self, other):
        #multiplication righthanded only (matrix*number)
        if isinstance(other, Matrix):
            assert self.width==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.zero(self.height,other.width,self[0,0]-self[0,0])
            for r in range(self.height):
                for c in range(other.width):
                    acc = None
                    for k in range(self.width):
                        add = self[r,k]*other[k,c]
                        acc = add if acc is None else acc+add
                    matrix[r,c] = acc
            return matrix
        elif isinstance(other, Number):
            matrix = self.zero(self.height,self.width,self[0,0]-self[0,0])
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c]=self[r,c]*other
            return matrix
        return NotImplemented
    
    def __truediv__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            divider = self.inverse()
            matrix = self*divider
            return matrix
        return NotImplemented
    
    def inverse(self):
        l, u = self.lu()
        null = self[0,0]-self[0,0]
        l_inv = l.empty_like()
        u_inv = u.empty_like()
        for i in range(self.height):
            for j in range(self.width):
                if i==j:
                    u_inv[i,i] = u.invert_element(u[i,i])
                    l_inv[i,j] = l.invert_element(l[i,j])
                elif i>j:
                    u_inv[i,j] = null
                    temp = null
                    for k in range(i):
                        temp += l_inv[k,j]*l[i,k]
                    l_inv[i,j] = -l.invert_element(l[i,i])*temp
                else:
                    l_inv[i,j] = null
                    temp = null
                    for k in range(j):
                        temp += u_inv[i,k]*u[k,j]
                    u_inv[i,j] = -u.invert_element(u[j,j])*temp
        return u_inv*l_inv

    def invert_element(self, element):
        if isinstance(element, Number):
            return 1/element
        if isinstance(element, Fraction):
            return 1/element
        if isinstance(element, Matrix):
            return element.inverse()
        raise TypeError

    def lu(self):
        raise NotImplementedError

    def det(self):
        assert self.width==self.height, f"Matrix is not square: {self.height} != {self.width}"
        l,u = self.lu()
        det = 1
        for i in range(u.height):
            det *= u[i,i]
        return det
    
    def lup(self):
        temp = self
        p = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            if self[i,i]!=0:
                p[i,i] = self[0,0]/self[0,0]
        for i in range(self.height):
            ref_val = 0
            ref_num = -1
            for j in range(i,self.width):
                if np.abs(temp[j,i])>=ref_val:
                    ref_val = np.abs(temp[j,i])
                    ref_num = j
            if ref_val!=0:
                temp.swap_rows(ref_num,i)
                p.swap_rows(ref_num,i)
                for j in range(i+1,self.height):
                    temp[j,i]/=temp[i,i]
                    for k in range(i+1,self.height):
                        temp[j,k] -= temp[j,i]*temp[i,k]
        u = self.zero(self.width,self.height,self[0,0]-self[0,0])
        l = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            for j in range(self.height):
                if i==j:
                    u[i,j] = temp[i,j]
                    l[i,j] = 1
                elif i<j:
                    u[i,j] = temp[i,j]
                    l[i,j] = 0
                else:
                    l[i,j] = temp[i,j]
                    u[i,j] = 0
        return temp,l,u,p
        
    def swap_rows(self, num1, num2):
        matrix = self
        temp = self.empty_like()
        for i in range(self.height):
            temp[num1, i] = self[num1, i]
            matrix[num1, i] = self[num2, i]
            matrix[num2, i] = temp[num1, i]
        return matrix 
    
    def swap_cols(self, num1, num2):
        matrix = self
        temp = self.empty_like()
        for i in range(self.width):
            temp[i, num1] = self[i, num1]
            matrix[i, num1] = self[i, num2]
            matrix[i, num2] = temp[i, num1]
        return matrix 
    
    def transpone(self):
        raise NotImplementedError

    def zero(self, width, height, param, low_bandw, upp_bandw):
        raise NotImplementedError

    def solve(self, vector):
        if isinstance(vector, Matrix):
            assert vector.width==1 and self.width==vector.height and self.height==self.width, f"Vector or matrix shape is wrong: {self.shape}, {vector.shape}"
            garbage, l, u, p = self.lup()
            y_vec = vector.empty_like()
            pb = p*vector
            result = vector.empty_like()
            temp = 0
            flag = 0
            #solving Ly=Pb
            for i in range(self.height):
                y_vec[i,0] = (pb[i,0] - temp)/l[i,i]
                temp = 0
                flag+=1
                for j in range(i+1):
                    if i<self.height-1:
                        temp += l[i+1,j]*y_vec[j,0]
            #solving Ux=y
            temp=0
            for i in range(self.height-1,-1,-1):
                result[i,0] = (y_vec[i,0] - temp)/u[i,i]
                temp = 0
                for j in range(self.height-1,i-1,-1):
                    temp += u[i-1,j]*result[j,0]
            return result
        return NotImplemented

class FullMatrix(Matrix):
    """
    Заполненная матрица с элементами произвольного типа.
    """
    def __init__(self, data):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data
        
    def transpone(self):
        matrix = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.width):
            for j in range(self.height):
                matrix[i,j]  = self[j,i]
        return matrix

    def empty_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]       
        data = np.empty((height,width), dtype=dtype)
        return FullMatrix(data)

    def ident_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]       
        data = np.empty((height,width), dtype=dtype)
        for i in range(self.height):
            data[i,i] = self[i,i]*self.invert_element(self[i,i])
        return FullMatrix(data)

    def lu(self):
        assert self.width==self.height, f"Matrix is not square: {self.height} != {self.width}"
        u = self.zero(self.width,self.height,self[0,0]-self[0,0])
        l = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            if self[i,i]!=0:
                l[i,i] = self[0,0]/self[0,0]
        for i in range(self.height):
            for j in range(self.height):
                if i<=j:
                    temp = u.zero(u.width,u.height,u[0,0]-u[0,0])
                    for k in range(i+1):
                        temp[i,j] = temp[i,j]+l[i,k]*u[k,j]
                    u[i,j] = self[i,j]-temp[i,j]
                elif i>j:
                    temp = u.zero(u.width,u.height,u[0,0]-u[0,0])
                    for k in range(j+1):
                        temp[i,j] = temp[i,j] + l[i,k]*u[k,j]
                    l[i,j] = (self[i,j]-temp[i,j])*u.invert_element(u[j,j])
        return l,u
        
    @classmethod
    def zero(_cls, height, width, default=0, low_bandw=0, upp_bandw=0):
        """
        Создает матрицу размера `width` x `height` со значениями по умолчанию `default`.
        """
        data = np.empty((height, width), dtype=type(default))
        data[:] = default
        return FullMatrix(data)
                    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def dtype(self):
        return self.data.dtype
        
    def __getitem__(self, key):
        row, column = key
        return self.data[row, column]
    
    def __setitem__(self, key, value):
        row, column = key
        self.data[row, column] = value

class SymmetricMatrix(Matrix):
    """
    Симметричная матрица
    """
    def __init__(self, data):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data

    def empty_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]
        data = np.empty((height,width), dtype=dtype)
        return SymmetricMatrix(data)

    def ident_like(self, width=None, height=None):
        data = self.empty_like()
        for i in range(self.height):
            data[i,i] = self[i,i]*self.invert_element(self[i,i])
            for j in range(self.height):
                if i!=j:
                    data[i,j] = self[0,0]-self[0,0]
        return data

    def lu(self):
        #cholecky decomposition used for this (l i not more uni-left-triangle)
        matrix = self.empty_like()
        for i in range(self.height):
            for j in range(i+1):
                temp = 0
                for k in range(j):
                    temp += matrix[i,k]*matrix[j,k]
                if i==j:
                    matrix[i,j] = np.sqrt(self[i,i]-temp)
                else:
                    matrix[i,j] = (self[i,j]-temp)*matrix.invert_element(matrix[j,j])
        l,u = FullMatrix.zero(self.width,self.height,self[0,0]-self[0,0]),FullMatrix.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            for j in range(self.width):
                if j>i:
                    l[i,j] = 0
                    u[i,j] = matrix[i,j]
                elif j<i:
                    u[i,j] = 0
                    l[i,j] = matrix[i,j]
                else:
                    u[i,j] = matrix[i,j]
                    l[i,j] = matrix[i,j]
        return l,u

    @classmethod
    def zero(_cls, width, height, default=0, low_bandw=0, upp_bandw=0):
        """
        Создает матрицу размера `height` x `height` со значениями по умолчанию `default`.
        """
        data = np.empty((height, height), dtype=type(default))
        data[:] = default
        return SymmetricMatrix(data)

    @property
    def shape(self):
        return self.data.shape

    @property
    def dtype(self):
        return self.data.dtype

    def __getitem__(self, key):
        row, column = key
        return self.data[row, column]

    def __setitem__(self, key, value):
        row, column = key
        self.data[row, column] = value
        self.data[column, row] = value

class BandMatrix(Matrix):
    """
    Заполненная матрица с элементами произвольного типа.
    """
    def __init__(self, data, low_bandw, upp_bandw):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data
        self.lw_bw = low_bandw
        self.up_bw = upp_bandw

    def transpone(self):
        matrix = self.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.width):
            for j in range(self.height):
                matrix[i,j]  = self[j,i]
        return matrix

    @classmethod
    def zero(_cls, height, width, default=0,low_bandw=0,upp_bandw=0):
        """
        Создает матрицу размера `width` x `height` со значениями по умолчанию `default`.
        """
        data = np.empty((height, low_bandw+upp_bandw+1), dtype=type(default))
        data[:] = default
        return BandMatrix(data,low_bandw,upp_bandw)

    @property
    def shape(self):
        real_shape = (self.data.shape[0],self.data.shape[0])
        return real_shape

    @property
    def dtype(self):
        return self.data.dtype

    @property
    def low_bandw(self):
        return self.lw_bw

    @property
    def upp_bandw(self):
        return self.up_bw

    def __getitem__(self, key):
        row, column = key
        if row < column-self.up_bw or row>column+self.lw_bw:
            return 0
        else:
            return self.data[row, self.upp_bandw+row-column-1]

    def __setitem__(self, key, value):
        row, column = key
        if row >= column-self.up_bw or row<=column+self.lw_bw:
            self.data[row, self.upp_bandw+row-column-1] = value

    def lu(self):
        assert self.width==self.height, f"Matrix is not square: {self.height} != {self.width}"
        u = FullMatrix.zero(self.width,self.height,self[0,0]-self[0,0])
        l = FullMatrix.zero(self.width,self.height,self[0,0]-self[0,0])
        for i in range(self.height):
            if self[i,i]!=0:
                l[i,i] = self[0,0]/self[0,0]
        for i in range(self.height):
            for j in range(self.height):
                if i<=j:
                    temp = 0
                    for k in range(i+1):
                        temp = temp+l[i,k]*u[k,j]
                    u[i,j] = self[i,j]-temp
                elif i>j:
                    temp = 0
                    for k in range(j+1):
                        temp = temp + l[i,k]*u[k,j]
                    l[i,j] = (self[i,j]-temp)*u.invert_element(u[j,j])
        assert l*u == self, f"LU decomposition is not complete"
        return l,u

In [191]:
m = BandMatrix.zero(5,5,0,0,0)
m[0,0]=10
m[1,1]=1
m[2,2]=2
m[3,3]=3
m[4,4]=15
print(m.data)
print("\n")
print(m)

[[10]
 [ 1]
 [ 2]
 [ 3]
 [15]]


| 10 0 0 0 0  |
| 0  1 0 0 0  |
| 0  0 2 0 0  |
| 0  0 0 3 0  |
| 0  0 0 0 15 |


In [192]:
m = BandMatrix.zero(5,5,0,0,0)
m[0,0]=10
m[1,1]=1
m[2,2]=2
m[3,3]=3
m[4,4]=15
m[3,4] = 7
m[2,4] = 9
m[1,0] = 8
m[0,4] = 10

print(m.data)
print("\n")
print(m)
print(m.lu())

IndexError: index -2 is out of bounds for axis 1 with size 1

In [165]:
m = FullMatrix.zero(3,5,0)
print(m)
print(m*5)
print(m.transpone())
print(m.shape)
print(m.dtype)
print(m.swap_rows(1,2))

| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 |
| 0 0 0 |
| 0 0 0 |
| 0 0 0 |
| 0 0 0 |
(3, 5)
int32
| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |


#### Невырожденные матрицы

In [166]:
m = FullMatrix.zero(3,3,Fraction(0,1))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)+i*i+j*(j+1)
d = FullMatrix.zero(3,3,Fraction(0,1))
for i in range(min(d.height,d.width)):
        d[i,i] = Fraction(i,1)        
print(m)
print(m.shape)
print(m.dtype)
print(f"m+m", m+m)
print(f"m*d", m*d)
print()
print("l=")
print(m.lu()[0])
print("u=")
print(m.lu()[1])
print("a=LU=")
print(m.lu()[0]*m.lu()[1])
print(m.det())
print(m.invert_element(m))

| 1 5/2  19/3 |
| 3 4    23/3 |
| 7 15/2 11   |
(3, 3)
object
m+m | 2  5  38/3 |
| 6  8  46/3 |
| 14 15 22   |
m*d | 0 5/2  38/3 |
| 0 4    46/3 |
| 0 15/2 22   |

l=
| 1 0    0 |
| 3 1    0 |
| 7 20/7 1 |
u=
| 1 5/2  19/3   |
| 0 -7/2 -34/3  |
| 0 0    -20/21 |
a=LU=
| 1 5/2  19/3 |
| 3 4    23/3 |
| 7 15/2 11   |
10/3
| -81/20 6   -37/20 |
| 31/5   -10 17/5   |
| -33/20 3   -21/20 |


### Блочные матрицы

In [167]:
matr = FullMatrix.zero(3,3,m)
matr[0,0]=m*m
matr[0,1]=m+m*m
matr[0,2]=m*m*m
matr[1,0]=m*m-m
matr[1,1]=m*3
matr[1,2]=m
matr[2,0]=m
matr[2,1]=m*2
matr[2,2]=m*3
print(matr)
print(matr.lu()[0])
print('\n',matr.lu()[1])
matr_new = matr.lu()[0]*matr.lu()[1]
print(matr_new)

| | 317/6 60  571/6  |   | 323/6 125/2 203/2  | | 899    6515/6 16573/9 | |
| | 206/3 81  134    |   | 215/3 85    425/3  | | 3749/3 4502/3 22769/9 | |
| | 213/2 130 1337/6 |   | 227/2 275/2 1403/6 | | 6169/3 4915/2 12367/3 | |
|                                                                         |
| | 311/6 115/2 533/6  | | 3  15/2 19 |         | 1 5/2  19/3 |           |
| | 197/3 77    379/3  | | 9  12   23 |         | 3 4    23/3 |           |
| | 199/2 245/2 1271/6 | | 21 45/2 33 |         | 7 15/2 11   |           |
|                                                                         |
| | 1 5/2  19/3 |        | 2  5  38/3 |         | 3  15/2 19 |            |
| | 3 4    23/3 |        | 6  8  46/3 |         | 9  12   23 |            |
| | 7 15/2 11   |        | 14 15 22   |         | 21 45/2 33 |            |
| | 1 0 0 |             | 0 0 0 |                               | 0 0 0 | |
| | 0 1 0 |             | 0 0 0 |                               | 0 0 0 | |
| | 0 0 1 | 

### LUP

In [169]:
m = FullMatrix.zero(4,4,Fraction(1,2))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)+i*i+j*j
print(m)
print("det",m.det())
temp,l,u,p = m.lup()
print("l")
print(l)
print("u")
print(u)
print("p")
print(p)
m_new = p.transpone()*l*u
print(m_new)

| 1  3/2  13/3 37/4 |
| 3  3    17/3 21/2 |
| 7  13/2 9    55/4 |
| 13 12   43/3 19   |
det 0
l
| 1    0    0 0 |
| 1/13 1    0 0 |
| 3/13 2/5  1 0 |
| 7/13 1/15 1 1 |
u
| 13 12    43/3  19     |
| 0  15/26 42/13 405/52 |
| 0  0     16/15 3      |
| 0  0     0     0      |
p
| 0 0 0 1 |
| 1 0 0 0 |
| 0 1 0 0 |
| 0 0 1 0 |
| 1  3/2  13/3 37/4 |
| 3  3    17/3 21/2 |
| 7  13/2 9    55/4 |
| 13 12   43/3 19   |


проверить матрицы с нулевым главным минором

In [171]:
m = FullMatrix.zero(3,3,Fraction(1,2))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+j+1)+i*i-j*j+i*j
b = FullMatrix.zero(3,1,Fraction(1,2))
for i in range(2):
    b[i,0] = i+1
print("b=",b)
print(m,"D=",m.det())
mb = m*b
print("mb=",mb)
print("\n",m.solve(b))


b= | 1   |
| 2   |
| 1/2 |
| 1 -2/3 -19/5 |
| 3 5/3  -3/5  |
| 7 6    23/5  | D= -4/5
mb= | -67/30 |
| 181/30 |
| 213/10 |

 | 745/24 |
| -195/4 |
| 395/24 |


## Симметричные матрицы

In [172]:
m = SymmetricMatrix.zero(3,3,Fraction)
m[1,2]=-43
m[0,0]=4
m[0,1]=12
m[0,2]=-16
m[1,1]=37
m[2,2]=98
print(m)
print(m.lu())
print(m.lu()[0]*m.lu()[1])

| 4   12  -16 |
| 12  37  -43 |
| -16 -43 98  |
(| 2  0 0 |
| 6  1 0 |
| -8 5 3 |, | 2 6 -8 |
| 0 1 5  |
| 0 0 3  |)
| 4   12  -16 |
| 12  37  -43 |
| -16 -43 98  |


### Матрица с нулевым минором

In [173]:
m = FullMatrix.zero(3,3,Fraction(0,24))
m[0,0] = 5
m[0,1] = 5
m[0,2] = 3
m[1,0] = 6
m[1,1] = 6
m[1,2] = 4
m[2,0] = 1
m[2,1] = 2
m[2,2] = 3
print(m)
print(m.lu())
print(m.lu()[0]*m.lu()[1])
print("\n",m)
g,l,u,p = m.lup()
print(p.transpone()*l*u)

| 5 5 3 |
| 6 6 4 |
| 1 2 3 |
(| 1 0 0 |
| 1 1 0 |
| 0 2 1 |, | 5 5 3 |
| 0 1 1 |
| 0 0 1 |)
| 5 5 3 |
| 5 6 4 |
| 0 2 3 |

 | 5 5 3 |
| 6 6 4 |
| 1 2 3 |
| 0 0 0 |
| 6 6 4 |
| 0 1 2 |


In [ ]:
def test_matrixes():
    #Full
    #Symmetric
    #Block
    #Band
    number_space = np.linspace(1,1e6,100)
    for n in number_space:
        m_full = FullMatrix.zero(n,n)
        m_symm = SymmetricMatrix.zero(n,n)
        m_block = FullMatrix.zero(n,n,m_full)
        for i in range(n):
            for j in range(n):
                val = np.random.randint(0,100,1)
                m_full[i,j] = val
                m_symm[i,j] = val

def test_lu():
